In [ ]:
#If not image file, convert from pdf to image 
from pdf2image import convert_from_path

def pdf_to_image(pdf_path):
    images = convert_from_path(pdf_path)
    return images[0]  # Assuming the chessboard is on the first page

In [ ]:
#preprocess image using OCR
import cv2

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)
    return thresh

In [ ]:
#detect chess pieces on chessboard
import pytesseract

def ocr_image(image):
    config = '--psm 6'  # Assume a single uniform block of text
    pieces_text = pytesseract.image_to_string(image, config=config)
    return pieces_text

In [ ]:
#translate OCR output to chess pgn

def parse_pieces(pieces_text):
    lines = pieces_text.strip().split("\n")
    board = [['' for _ in range(8)] for _ in range(8)]
    
    for line in lines:
        # Parse each line to get the piece and its position
        piece, position = line[0], line[1:]  # Simplified for example
        col = ord(position[0].lower()) - ord('a')
        row = int(position[1]) - 1
        board[row][col] = piece

    return board

In [ ]:
import chess
import chess.pgn

def generate_pgn(board):
    game = chess.pgn.Game()
    node = game

    for row in range(8):
        for col in range(8):
            piece = board[row][col]
            if piece:
                node = node.add_variation(chess.Move.from_uci(f"{chr(col + ord('a'))}{row + 1}"))

    return game

# Example usage
def main(file_path):
    if file_path.endswith('.pdf'):
        image = pdf_to_image(file_path)
    else:
        image = cv2.imread(file_path)

    preprocessed_image = preprocess_image(image)
    pieces_text = ocr_image(preprocessed_image)
    board = parse_pieces(pieces_text)
    game = generate_pgn(board)
    print(game)

# main("path_to_your_file")